In [1]:
#pip install nltk
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [29]:
import nltk
import random
import nltk.corpus
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
import pickle
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity



nltk.download('movie_reviews')



[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\IITC\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [40]:
print( movie_reviews)

<CategorizedPlaintextCorpusReader in 'C:\\Users\\IITC\\AppData\\Roaming\\nltk_data\\corpora\\movie_reviews'>


In [39]:
print( movie_reviews.categories())
print( movie_reviews.fileids('neg')[:2])
print( movie_reviews.words('neg/cv000_29416.txt'))


['neg', 'pos']
['neg/cv000_29416.txt', 'neg/cv001_19502.txt']
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]


In [4]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]


In [47]:
#print(documents[:1])
print(documents[0][0])
print(documents[0][1])

['for', 'a', 'film', 'touted', 'as', 'exploring', 'relationships', 'and', 'black', 'sexuality', ',', 'trois', 'is', 'surprisingly', 'tame', '.', 'despite', 'it', "'", 's', 'lurid', 'subject', 'matter', 'and', 'it', "'", 's', 'passing', 'nod', 'to', 'fatal', 'attraction', ',', 'it', 'moves', 'along', 'with', 'flat', ',', 'uninspired', 'dialogue', 'as', 'it', 'sets', 'up', 'a', 'surprising', 'climax', 'that', 'tries', 'mightily', 'to', 'overthrow', 'the', 'considerable', 'dead', 'weight', 'of', 'the', 'rest', 'of', 'the', 'film', '.', 'freshly', 'moved', 'to', 'atlanta', ',', 'jermaine', '(', 'dourdan', ')', 'and', 'his', 'wife', ',', 'jasmine', '(', 'moore', ')', ',', 'have', 'the', 'trappings', 'of', 'a', 'perfect', 'life', '.', 'they', 'have', 'a', 'beautiful', 'house', 'in', 'suburbia', '.', 'jermaine', 'is', 'a', 'lawyer', 'on', 'the', 'fast', 'track', 'at', 'his', 'firm', '.', 'jasmine', 'is', 'his', 'supportive', 'wife', ',', 'who', 'is', 'finishing', 'up', 'her', 'college', 'degr

In [8]:

random.shuffle(documents)

all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

In [10]:
print(word_features[:10])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party']


In [11]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [12]:
#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))


{'plot': True, ':': True, 'two': True, 'teen': True, 'couples': True, 'go': True, 'to': True, 'a': True, 'church': True, 'party': True, ',': True, 'drink': True, 'and': True, 'then': True, 'drive': True, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': True, 'one': True, 'of': True, 'the': True, 'guys': True, 'dies': True, 'but': True, 'his': True, 'girlfriend': True, 'continues': True, 'see': True, 'him': True, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': True, 'what': True, "'": True, 's': True, 'deal': True, '?': True, 'watch': True, 'movie': True, '"': True, 'sorta': True, 'find': True, 'out': True, 'critique': True, 'mind': True, '-': True, 'fuck': True, 'for': True, 'generation': True, 'that': True, 'touches': True, 'on': True, 'very': True, 'cool': True, 'idea': True, 'presents': True, 'it': True, 'bad': True, 'package': True, 'which': True, 'is': True, 'makes': True, 'this': True, 'review': True, 'even': True, 'harder': True, 'wri

In [13]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]


In [14]:
# set that we'll train our classifier with
training_set = featuresets[:1900]

# set that we'll test against.
testing_set = featuresets[1900:]

In [16]:
print(training_set[:1])

[({'plot': False, ':': True, 'two': False, 'teen': False, 'couples': False, 'go': False, 'to': True, 'a': True, 'church': False, 'party': False, ',': True, 'drink': False, 'and': True, 'then': False, 'drive': False, '.': True, 'they': True, 'get': False, 'into': True, 'an': True, 'accident': False, 'one': True, 'of': True, 'the': True, 'guys': False, 'dies': False, 'but': True, 'his': True, 'girlfriend': False, 'continues': False, 'see': True, 'him': False, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': False, 'what': False, "'": True, 's': True, 'deal': False, '?': True, 'watch': False, 'movie': True, '"': True, 'sorta': False, 'find': True, 'out': True, 'critique': False, 'mind': False, '-': True, 'fuck': False, 'for': True, 'generation': False, 'that': True, 'touches': False, 'on': True, 'very': False, 'cool': False, 'idea': False, 'presents': False, 'it': True, 'bad': False, 'package': False, 'which': False, 'is': True, 'makes': False, 'this': True, 'review': Fal

In [17]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [18]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 85.0


In [19]:
classifier.show_most_informative_features(15)

Most Informative Features
                   sucks = True              neg : pos    =     10.3 : 1.0
                  annual = True              pos : neg    =      9.6 : 1.0
                 frances = True              pos : neg    =      8.9 : 1.0
                  stinks = True              neg : pos    =      8.4 : 1.0
             silverstone = True              neg : pos    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.5 : 1.0
              schumacher = True              neg : pos    =      7.4 : 1.0
           unimaginative = True              neg : pos    =      7.0 : 1.0
               atrocious = True              neg : pos    =      6.6 : 1.0
                  turkey = True              neg : pos    =      6.4 : 1.0
                 kidding = True              neg : pos    =      6.4 : 1.0
                 cunning = True              pos : neg    =      6.3 : 1.0
                 singers = True              pos : neg    =      6.3 : 1.0

In [22]:
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [23]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [26]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, testing_set))

BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)
print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, testing_set))

MultinomialNB accuracy percent: 0.87
BernoulliNB accuracy percent: 0.86


In [28]:
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


Original Naive Bayes Algo accuracy percent: 85.0
Most Informative Features
                   sucks = True              neg : pos    =     10.3 : 1.0
                  annual = True              pos : neg    =      9.6 : 1.0
                 frances = True              pos : neg    =      8.9 : 1.0
                  stinks = True              neg : pos    =      8.4 : 1.0
             silverstone = True              neg : pos    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.5 : 1.0
              schumacher = True              neg : pos    =      7.4 : 1.0
           unimaginative = True              neg : pos    =      7.0 : 1.0
               atrocious = True              neg : pos    =      6.6 : 1.0
                  turkey = True              neg : pos    =      6.4 : 1.0
                 kidding = True              neg : pos    =      6.4 : 1.0
                 cunning = True              pos : neg    =      6.3 : 1.0
                 singers 

C:\Users\IITC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression_classifier accuracy percent: 90.0
SGDClassifier_classifier accuracy percent: 90.0


C:\Users\IITC\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC_classifier accuracy percent: 85.0
LinearSVC_classifier accuracy percent: 88.0


C:\Users\IITC\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


NuSVC_classifier accuracy percent: 93.0


In [61]:

doc_df=pd.DataFrame(documents)
doc_df.columns = ['text', 'class']


doc_df.head



<bound method NDFrame.head of                                                    text class
0     [for, a, film, touted, as, exploring, relation...   neg
1     [`, the, bachelor, ', is, one, of, the, best, ...   neg
2     [synopsis, :, retiring, detective, jerry, blac...   pos
3     [deserves, recognition, for, :, making, this, ...   neg
4     [in, recent, years, ,, harrison, ford, has, be...   pos
...                                                 ...   ...
1995  [carolco, pictures, and, dutch, director, paul...   pos
1996  [there, ', s, a, good, ,, and, timely, ,, stor...   neg
1997  [in, these, days, of, overlong, movies, (, mee...   pos
1998  [stephen, ,, please, post, if, appropriate, .,...   neg
1999  [alchemy, is, steeped, in, shades, of, blue, ....   pos

[2000 rows x 2 columns]>

In [64]:
doc_df['text' ].head

<bound method NDFrame.head of 0       [for, a, film, touted, as, exploring, relation...
1       [`, the, bachelor, ', is, one, of, the, best, ...
2       [synopsis, :, retiring, detective, jerry, blac...
3       [deserves, recognition, for, :, making, this, ...
4       [in, recent, years, ,, harrison, ford, has, be...
                              ...                        
1995    [carolco, pictures, and, dutch, director, paul...
1996    [there, ', s, a, good, ,, and, timely, ,, stor...
1997    [in, these, days, of, overlong, movies, (, mee...
1998    [stephen, ,, please, post, if, appropriate, .,...
1999    [alchemy, is, steeped, in, shades, of, blue, ....
Name: text, Length: 2000, dtype: object>

In [65]:
doc_df['class'].head

<bound method NDFrame.head of 0       neg
1       neg
2       pos
3       neg
4       pos
       ... 
1995    pos
1996    neg
1997    pos
1998    neg
1999    pos
Name: class, Length: 2000, dtype: object>

In [66]:

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')


In [67]:
doc_df['Joined'] = doc_df.text.apply(', '.join)



In [68]:
doc_df['Joined'].head

<bound method NDFrame.head of 0       for, a, film, touted, as, exploring, relations...
1       `, the, bachelor, ', is, one, of, the, best, t...
2       synopsis, :, retiring, detective, jerry, black...
3       deserves, recognition, for, :, making, this, r...
4       in, recent, years, ,, harrison, ford, has, bee...
                              ...                        
1995    carolco, pictures, and, dutch, director, paul,...
1996    there, ', s, a, good, ,, and, timely, ,, story...
1997    in, these, days, of, overlong, movies, (, meet...
1998    stephen, ,, please, post, if, appropriate, ., ...
1999    alchemy, is, steeped, in, shades, of, blue, .,...
Name: Joined, Length: 2000, dtype: object>

In [80]:
doc_df['id'] =doc_df.index

In [81]:
doc_df.head

<bound method NDFrame.head of                                                    text class  \
0     [for, a, film, touted, as, exploring, relation...   neg   
1     [`, the, bachelor, ', is, one, of, the, best, ...   neg   
2     [synopsis, :, retiring, detective, jerry, blac...   pos   
3     [deserves, recognition, for, :, making, this, ...   neg   
4     [in, recent, years, ,, harrison, ford, has, be...   pos   
...                                                 ...   ...   
1995  [carolco, pictures, and, dutch, director, paul...   pos   
1996  [there, ', s, a, good, ,, and, timely, ,, stor...   neg   
1997  [in, these, days, of, overlong, movies, (, mee...   pos   
1998  [stephen, ,, please, post, if, appropriate, .,...   neg   
1999  [alchemy, is, steeped, in, shades, of, blue, ....   pos   

                                                 Joined    id  
0     for, a, film, touted, as, exploring, relations...     0  
1     `, the, bachelor, ', is, one, of, the, best, t...     1

In [69]:
tfidf_matrix = tf.fit_transform(doc_df['Joined'])

In [77]:
print("The Dim of tfidf_matrix is: ", (tfidf_matrix.shape)) 
print(tfidf_matrix)


The Dim of tfidf_matrix is:  (2000, 1150395)
  (0, 792967)	0.037764783373837596
  (0, 1055245)	0.037764783373837596
  (0, 963017)	0.037764783373837596
  (0, 400366)	0.037764783373837596
  (0, 310525)	0.037764783373837596
  (0, 293573)	0.037764783373837596
  (0, 677516)	0.037764783373837596
  (0, 847640)	0.037764783373837596
  (0, 137477)	0.037764783373837596
  (0, 776596)	0.037764783373837596
  (0, 225131)	0.037764783373837596
  (0, 1031770)	0.037764783373837596
  (0, 1070393)	0.037764783373837596
  (0, 661853)	0.037764783373837596
  (0, 626)	0.037764783373837596
  (0, 640180)	0.037764783373837596
  (0, 511211)	0.037764783373837596
  (0, 13261)	0.037764783373837596
  (0, 502100)	0.037764783373837596
  (0, 631047)	0.037764783373837596
  (0, 277633)	0.037764783373837596
  (0, 30404)	0.037764783373837596
  (0, 914818)	0.037764783373837596
  (0, 1007862)	0.037764783373837596
  (0, 77386)	0.037764783373837596
  :	:
  (1999, 151715)	0.029823930041107196
  (1999, 378764)	0.013180207844785258


In [78]:
cosine_similarities = linear_kernel(tfidf_matrix)

print("The Dim of cosine_similarities is: ", (cosine_similarities.shape)) 
print(cosine_similarities)


results = {}


The Dim of cosine_similarities is:  (2000, 2000)
[[1.         0.0075309  0.00447953 ... 0.00491013 0.00285701 0.00299199]
 [0.0075309  1.         0.00335792 ... 0.00732916 0.00469899 0.00335505]
 [0.00447953 0.00335792 1.         ... 0.00172811 0.00611233 0.00358856]
 ...
 [0.00491013 0.00732916 0.00172811 ... 1.         0.00443289 0.00414453]
 [0.00285701 0.00469899 0.00611233 ... 0.00443289 1.         0.00349114]
 [0.00299199 0.00335505 0.00358856 ... 0.00414453 0.00349114 1.        ]]


In [82]:

for idx, row in doc_df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], doc_df['id'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
    
print('done!')


done!


In [84]:
def item(id):
    return doc_df.loc[doc_df['id'] == id]['description'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + doc_df(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")



In [85]:
recommend(item_id=11, num=5)

TypeError: 'DataFrame' object is not callable